# What's Cooking  in Python
https://www.kaggle.com/manuelatadvice/whats-cooking/noname/code

In [56]:

#This imports a bunch of packages.  
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.stem import WordNetLemmatizer
from collections import Counter
import json
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import grid_search


In [4]:

#If you import the codes locally, this seems to cause some issues.  
import json
from urllib.request import urlopen

urltrain= 'https://raw.githubusercontent.com/RPI-Analytics/MGMT6963-2015/master/data/whatscooking/whatscookingtrain.json'
urltest = 'https://raw.githubusercontent.com/RPI-Analytics/MGMT6963-2015/master/data/whatscooking/whatscookingtest.json'


train = pd.read_json(urlopen(urltrain))
test = pd.read_json(urlopen(urltest))

In [5]:
#First we want to see the most popular cuisine for the naive model. 
train.groupby('cuisine').size()

cuisine
brazilian        467
british          804
cajun_creole    1546
chinese         2673
filipino         755
french          2646
greek           1175
indian          3003
irish            667
italian         7838
jamaican         526
japanese        1423
korean           830
mexican         6438
moroccan         821
russian          489
southern_us     4320
spanish          989
thai            1539
vietnamese       825
dtype: int64

In [6]:
#Here we write the most popular selction.  This is the baseline by which we will judge other models. 
test['cuisine']='italian'

In [7]:
#THis is a much more simple version that selects out the columns ID and cuisinte
submission=test[['id' ,  'cuisine' ]]
#This is a more complex method I showed that gives same.
#submission=pd.DataFrame(test.ix[:,['id' ,  'cuisine' ]])

In [8]:
#This outputs the file.
submission.to_csv("1_cookingSubmission.csv",index=False)


In [9]:
#So it seems there is some data we need to use the NLTK leemmatizer.  
stemmer = WordNetLemmatizer()
nltk.download('wordnet')

 

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [33]:
#We see this in a Python Solution. 
train['ingredients_clean_string1'] = [' , '.join(z).strip() for z in train['ingredients']] 

#We also know that we can do something similar though a Lambda function. 
strip = lambda x: ' , '.join(x).strip() 
#Finally, we call the function for name
train['ingredients_clean_string2'] = train['ingredients'].map(strip)

#Now that we used the lambda function, we can reuse this for the test dataset. 
test['ingredients_clean_string1'] = test['ingredients'].map(strip)
 


In [34]:
#We see this in one of the solutions.  We can reconstruct it in a way that makes it abit easier to follow, but I found when doing that it took forever.  

#To interpret this, read from right to left. 
train['ingredients_string1'] = [' '.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in train['ingredients']]       



In [38]:
#Now we will create a corpus.
corpustr = train['ingredients_string1']
corpusts = test['ingredients_string1']


In [43]:
#http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
#You could develop an understanding based on each.  
vectorizertr = TfidfVectorizer(stop_words='english',
                             ngram_range = ( 1 , 1 ),analyzer="word", 
                             max_df = .57 , binary=False , token_pattern=r'\w+' , sublinear_tf=False)
vectorizerts = TfidfVectorizer(stop_words='english')

In [48]:
#Note that this doesn't work with the #todense option.  
tfidftr=vectorizertr.fit_transform(corpustr)
predictors_tr = tfidftr

In [61]:
#Note that this doesn't work with the #todense option.  This creates a matrix of predictors from the corpus. 
tfidfts=vectorizertr.transform(corpusts)
predictors_ts= tfidfts

<9944x2963 sparse matrix of type '<class 'numpy.float64'>'
	with 182346 stored elements in Compressed Sparse Row format>

In [50]:
#This is target variable.  
targets_tr = train['cuisine']


In [53]:
#Logistic Regression. 
parameters = {'C':[1, 10]}
#clf = LinearSVC()
clf = LogisticRegression()



In [57]:
#This uses that associated paramters to search a grid space. 
classifier = grid_search.GridSearchCV(clf, parameters)
classifier=classifier.fit(predictors_tr,targets_tr)



In [62]:
#This predicts the outcome for the test set. 
predictions=classifier.predict(predictors_ts)

In [64]:
#This adds it to the resulting dataframe. 
test['cuisine'] = predictions

In [65]:
#This creates the submision dataframe
submission2=test[['id' ,  'cuisine' ]]

In [66]:
#This outputs the file.
submission2.to_csv("2_logisticSubmission.csv",index=False)

In [67]:
from sklearn.ensemble import RandomForestClassifier 



In [1]:
# Create the random forest object which will include all the parameters
# for the fit
forest = RandomForestClassifier(n_estimators = 10)



NameError: name 'RandomForestClassifier' is not defined

In [ ]:
# Fit the training data to the Survived labels and create the decision trees
forest = forest.fit(predictors_tr,targets_tr)



In [ ]:
# Take the same decision trees and run it on the test data
predictions = forest.predict(predictors_ts)

In [ ]:
#This adds it to the resulting dataframe. 
test['cuisine'] = predictions

In [ ]:
#This creates the submision dataframe
submission2=test[['id' ,  'cuisine' ]]